In [ ]:
import pandas as pd

covid_data = pd.read_csv('/content/covid_data.csv', delimiter=';')

<ipython-input-8-93389f6a1533>:3: DtypeWarning: Columns (16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  covid_data = pd.read_csv('/content/covid_data.csv', delimiter=';')


In [ ]:
us_data = covid_data[covid_data['Country_Region'] == 'US']
us_data['Last_Update'] = pd.to_datetime(us_data['Last_Update'], format='mixed')
us_data = us_data.sort_values(by='Last_Update')
us_data = us_data[['Province_State','Admin2','Last_Update', 'Confirmed', 'Deaths']]

us_data

<ipython-input-96-78f2f72f5212>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us_data['Last_Update'] = pd.to_datetime(us_data['Last_Update'], format='mixed')


,Province_State,Admin2,Last_Update,Confirmed,Deaths
177446,Alabama,Clay,2020-03-22 23:45:00,0.0,0.0
177574,California,Sutter,2020-03-22 23:45:00,0.0,0.0
177575,California,Tehama,2020-03-22 23:45:00,0.0,0.0
177576,California,Trinity,2020-03-22 23:45:00,0.0,0.0
177577,California,Tuolumne,2020-03-22 23:45:00,0.0,0.0
...,...,...,...,...,...
17597,Kentucky,Mason,2023-03-10 04:21:03,6323.0,118.0
17598,Kentucky,McCracken,2023-03-10 04:21:03,23983.0,313.0
17599,Kentucky,McCreary,2023-03-10 04:21:03,7724.0,91.0
17589,Kentucky,Livingston,2023-03-10 04:21:03,3104.0,67.0


In [ ]:
us_data['Admin2'] = us_data['Admin2'].fillna('Statewide')  # Nom arbitraire pour signaler l'absence d'Admin2

us_data_last = us_data.drop_duplicates(subset=['Admin2', 'Province_State'], keep='last')

us_state_summary = us_data_last.groupby('Province_State').agg({
    'Confirmed': 'sum',
    'Deaths': 'sum'
}).reset_index()

state_codes = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
    'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA',
    'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA',
    'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO',
    'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
    'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND',
    'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI',
    'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT',
    'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

us_state_summary['code'] = us_state_summary['Province_State'].map(state_codes)

us_state_summary

,Province_State,Confirmed,Deaths,code
0,Alabama,1644533.0,21032.0,AL
1,Alaska,307657.0,1486.0,AK
2,American Samoa,8320.0,34.0,NaN
3,Arizona,2443514.0,33102.0,AZ
4,Arkansas,1006883.0,13020.0,AR
5,California,12129699.0,101159.0,CA
6,Colorado,1764401.0,14181.0,CO
7,Connecticut,976657.0,12220.0,CT
8,Delaware,330793.0,3324.0,DE
9,Diamond Princess,49.0,0.0,NaN


In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Choropleth(
    locations=us_state_summary['code'], # Spatial coordinates
    z = us_state_summary['Deaths'].astype(float), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Reds',
    colorbar_title = "Deaths",
))

fig.update_layout(
    title_text = 'COVID-19 Deaths in the USA by State',
    geo_scope='usa', # limite map scope to USA
)

fig.show()

In [ ]:
import plotly.graph_objects as go
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')

fig = go.Figure(data=go.Choropleth(
    locations = df['CODE'],
    z = df['GDP (BILLIONS)'],
    text = df['COUNTRY'],
    colorscale = 'Blues',
    autocolorscale=False,
    reversescale=True,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_tickprefix = '$',
    colorbar_title = 'GDP<br>Billions US$',
))

fig.update_layout(
    title_text='2014 Global GDP',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    annotations = [dict(
        x=0.55,
        y=0.1,
        xref='paper',
        yref='paper',
        text='Source: <a href="https://www.cia.gov/library/publications/the-world-factbook/fields/2195.html">\
            CIA World Factbook</a>',
        showarrow = False
    )]
)

fig.show()

In [ ]:
df

,COUNTRY,GDP (BILLIONS),CODE
0,Afghanistan,21.71,AFG
1,Albania,13.40,ALB
2,Algeria,227.80,DZA
3,American Samoa,0.75,ASM
4,Andorra,4.80,AND
...,...,...,...
217,Virgin Islands,5.08,VGB
218,West Bank,6.64,WBG
219,Yemen,45.45,YEM
220,Zambia,25.61,ZMB
